In [1]:
import pandas as pd
import numpy as np

In [2]:
record_id = 1
drive_id = 1

In [3]:
# use .format(record_id, drive_id) for record X and drive Y
video_path = "./record{}/drive{}/video_camera1.mp4"

In [4]:
# use .format(record_id, drive_id, filename) for record X and drive Y and processed data filename
data_info_path = "./record{}/drive{}/processed_data/detection_camera1/{}"

* **detections.txt** ->
provides bounding boxes and contours of objects in the scene automatically generated by a CNN.
<br>
<br>
* **detectionsfiltered.txt** ->
are the same detections with a minimum confidence value of 0.5 and non-maximal suppression.
<br>
<br>
* **detectionstracked.txt** ->
is the result of a temporal tracking of the filtered detections, assigning a uniqueid to each object along the frames. Data is stored as a sequence of [frame,id,class,xi,yi,xf,yf,conf,n] values followed by n tuples of x and y coordinates that represents the contour.
<br>
<br>
* **labels.txt** -> is  a  sequence  of [frame,id,x,y,width,height] values for each manual annotation that describes an area centered in the vehicle.
<br>
<br>
* **lanes.txt** -> is  a  sequence  of [frame, laneID, c0, c1, c2, c3, isDetected, isPermanent, counterThreshold] values that represents the n lane lines in the scene as a 2nd order polynomial (Eq. 4).

* c0: lateral offset
* c1: angular error
* c2: curvature
* c3: curvature derivative
* isDetected: value 1 if the lane has been detected in the corresponding frame
* isPermanent: value 1 if if the lane has been maintained for at least 10 frames
* counterThreshold: represents the count of frames up or down used to set a threshold to set the lanes (defined at 10 frames) to activate and deactivate the isPermanent flag for that laneID
<br>
<br>
* **lanechange.txt** ->  ***correction* data type: [id_l, id_d, type, f0, fe, ff, blk] f0: event start, fe: event, ff: event finish (as frame number)** 

is a list of four values [id,type,frame,val1,val2] that indicates a type of lane change that  could  be left(1)  or right(2)  performed  by  the vehicle  with  that  id  at  that  frame.  The  parameter val1 is  used  for  timelapse  events  annotations,  such  as  beginning  of  the  lane  changes.  Lane  change  can  be  also labeled as cut-in(1), cut-out(2) when the vehicle arrives to  or  leaves  the  ego-lane  or none(0)  otherwise.  This information is codified in val2. Moreover, some anomalous  circumstances  are  labeled  ashazardous(3),  e.g.extreme lane changes, stopped vehicles on the shoulder, or  emergency  vehicle  overtaking.  These  situations  are maintained in the time and are labeled as the initial and final  frame  making  use  of  the val1 parameter.  Events related with pedestrians such as zebra crossing are also labeled (4) defining the frame when the pedestrian starts to  cross  or  he/she  is  seen  for  the  first  time.  The  end of  the  zebra  crossing  event  is  defined  using  parameter val1.

if lc_type: %d for id_d (vehicle id)
1. "cut-in by id %d"
2. "cut-out by id %d"
3. "left-ll by id %d"
4. "right-ll by id %d"
5. "hazardous by id %d"
6. "pedestrian by id %d"
<br>
<br>
* **trajectories.txt** -> is  a  sequence  of [frame,id,xc,yc,zc,xl,yl,zl] values that represents the position of  a  vehicle  in  the  camera  and  the  LiDAR  reference system.


### Print lane lines and masks on frame
1. read lanes.txt 
2. get each line data of that frame to a list **if detected**
3. print lane lines on the frame image
<br>
<br>
4. read detections_tracked.txt
5. get each detection masks to a list
6. print masks on the frame image

7. save the image

In [5]:
# [frame, laneID, c0, c1, c2, c3, isDetected, isPermanent, counterThreshold]
lanes_df = pd.read_csv(data_info_path.format(record_id, drive_id, "lanes.txt"), sep=" ",
                              names=["frame_id", "lane_id", "c0", "c1", "c2", "c3", "is_detected", "is_permanent", "counter_threshold"])

In [6]:
lanes_df.head(5)

,frame_id,lane_id,c0,c1,c2,c3,is_detected,is_permanent,counter_threshold
0,0,0,-0.896,-0.04,0.0,0.0,1,0,1
1,0,1,0.000,0.00,0.0,0.0,0,0,0
2,0,2,0.000,0.00,0.0,0.0,0,0,0
3,0,3,0.000,0.00,0.0,0.0,0,0,0
4,0,4,0.000,0.00,0.0,0.0,0,0,0


In [7]:
# [frame id class xi yi xf yf conf n nContourValues]
detections_tracked_df = pd.read_csv(data_info_path.format(record_id, drive_id, "detections_tracked.txt"), header=None, sep='\n')
detections_tracked_df = detections_tracked_df[0].str.split(' ', expand=True)

In [8]:
#detections_tracked_df = detections_tracked_df.iloc[:,:9]
detections_tracked_df.rename(columns = {0:"frame", 1:"veh_id", 2:"veh_class", 3:"xi", 4:"yi", 5:"xf", 6:"yf", 7:"conf", 8:"n_count"}, inplace=True)

In [9]:
detections_tracked_df.head(5)

,frame,veh_id,veh_class,xi,yi,xf,yf,conf,n_count,9,...,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680
0,000002,-00002,3,1582.2,0086.8,1770.4,0223.5,0.998,000189,1651,...,None,None,None,None,None,None,None,None,None,None
1,000002,-00003,3,1781.4,0085.4,1804.9,0107.2,0.536,000019,1782,...,None,None,None,None,None,None,None,None,None,None
2,000002,-00004,3,1734.9,0078.9,1780.9,0134.8,0.905,000062,1738,...,None,None,None,None,None,None,None,None,None,None
3,000003,-00002,3,1582.2,0086.5,1770.9,0223.3,0.998,000194,1652,...,None,None,None,None,None,None,None,None,None,None
4,000003,-00003,3,1781.4,0085.4,1804.9,0107.2,0.530,000023,1784,...,None,None,None,None,None,None,None,None,None,None


In [10]:
def get_contour_pts(row):
    mask_count = int(row[1]["n_count"]) * 2
    masks = row[1][9:mask_count+9].to_list()
    contours = []
    for i in range(0,len(masks),2):
        contours.append([int(masks[i]),int(masks[i+1])])
    return np.array(contours)

In [11]:
color_dict = {}
def create_color(veh_id):
    color = tuple(np.random.randint(256, size=3))
    color_dict[veh_id] = color
    return color

In [12]:
import cv2

cap = cv2.VideoCapture(video_path.format(record_id, drive_id))
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)

out = cv2.VideoWriter('test.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
count = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_id = '{:06}'.format(count)
        print(frame_id)
        #from IPython.core.debugger import set_trace
        #set_trace()
        
        rows = detections_tracked_df.loc[detections_tracked_df['frame'] == frame_id]
        if rows.empty == False:
            for row in rows.iterrows():
                #from IPython.core.debugger import set_trace
                #set_trace()
                veh_id = row[1]["veh_id"]

                # assign a color to that veh_id
                if veh_id in color_dict:
                    color = color_dict[veh_id]
                else:
                    color = create_color(veh_id)

                # get contour points
                cntr_pts = get_contour_pts(row)

                # draw contour on that specific frame
                cv2.fillPoly(frame, pts=[cntr_pts], color=(int(color[0]), int(color[1]), int(color[2])))

        out.write(frame)
        count += 1
    else:
        break
        
cap.release()
out.release()

000000
000001
000002
000003
000004
000005
000006
000007
000008
000009
000010
000011
000012
000013
000014
000015
000016
000017
000018
000019
000020
000021
000022
000023
000024
000025
000026
000027
000028
000029
000030
000031
000032
000033
000034
000035
000036
000037
000038
000039
000040
000041
000042
000043
000044
000045
000046
000047
000048
000049
000050
000051
000052
000053
000054
000055
000056
000057
000058
000059
000060
000061
000062
000063
000064
000065
000066
000067
000068
000069
000070
000071
000072
000073
000074
000075
000076
000077
000078
000079
000080
000081
000082
000083
000084
000085
000086
000087
000088
000089
000090
000091
000092
000093
000094
000095
000096
000097
000098
000099
000100
000101
000102
000103
000104
000105
000106
000107
000108
000109
000110
000111
000112
000113
000114
000115
000116
000117
000118
000119
000120
000121
000122
000123
000124
000125
000126
000127
000128
000129
000130
000131
000132
000133
000134
000135
000136
000137
000138
000139
000140
000141
000142

001175
001176
001177
001178
001179
001180
001181
001182
001183
001184
001185
001186
001187
001188
001189
001190
001191
001192
001193
001194
001195
001196
001197
001198
001199
001200
001201
001202
001203
001204
001205
001206
001207
001208
001209
001210
001211
001212
001213
001214
001215
001216
001217
001218
001219
001220
001221
001222
001223
001224
001225
001226
001227
001228
001229
001230
001231
001232
001233
001234
001235
001236
001237
001238
001239
001240
001241
001242
001243
001244
001245
001246
001247
001248
001249
001250
001251
001252
001253
001254
001255
001256
001257
001258
001259
001260
001261
001262
001263
001264
001265
001266
001267
001268
001269
001270
001271
001272
001273
001274
001275
001276
001277
001278
001279
001280
001281
001282
001283
001284
001285
001286
001287
001288
001289
001290
001291
001292
001293
001294
001295
001296
001297
001298
001299
001300
001301
001302
001303
001304
001305
001306
001307
001308
001309
001310
001311
001312
001313
001314
001315
001316
001317

002346
002347
002348
002349
002350
002351
002352
002353
002354
002355
002356
002357
002358
002359
002360
002361
002362
002363
002364
002365
002366
002367
002368
002369
002370
002371
002372
002373
002374
002375
002376
002377
002378
002379
002380
002381
002382
002383
002384
002385
002386
002387
002388
002389
002390
002391
002392
002393
002394
002395
002396
002397
002398
002399
002400
002401
002402
002403
002404
002405
002406
002407
002408
002409
002410
002411
002412
002413
002414
002415
002416
002417
002418
002419
002420
002421
002422
002423
002424
002425
002426
002427
002428
002429
002430
002431
002432
002433
002434
002435
002436
002437
002438
002439
002440
002441
002442
002443
002444
002445
002446
002447
002448
002449
002450
002451
002452
002453
002454
002455
002456
002457
002458
002459
002460
002461
002462
002463
002464
002465
002466
002467
002468
002469
002470
002471
002472
002473
002474
002475
002476
002477
002478
002479
002480
002481
002482
002483
002484
002485
002486
002487
002488

003517
003518
003519
003520
003521
003522
003523
003524
003525
003526
003527
003528
003529
003530
003531
003532
003533
003534
003535
003536
003537
003538
003539
003540
003541
003542
003543
003544
003545
003546
003547
003548
003549
003550
003551
003552
003553
003554
003555
003556
003557
003558
003559
003560
003561
003562
003563
003564
003565
003566
003567
003568
003569
003570
003571
003572
003573
003574
003575
003576
003577
003578
003579
003580
003581
003582
003583
003584
003585
003586
003587
003588
003589
003590
003591
003592
003593
003594
003595
003596
003597
003598
003599
003600
003601
003602
003603
003604
003605
003606
003607
003608
003609
003610
003611
003612
003613
003614
003615
003616
003617
003618
003619
003620
003621
003622
003623
003624
003625
003626
003627
003628
003629
003630
003631
003632
003633
003634
003635
003636
003637
003638
003639
003640
003641
003642
003643
003644
003645
003646
003647
003648
003649
003650
003651
003652
003653
003654
003655
003656
003657
003658
003659

004690
004691
004692
004693
004694
004695
004696
004697
004698
004699
004700
004701
004702
004703
004704
004705
004706
004707
004708
004709
004710
004711
004712
004713
004714
004715
004716
004717
004718
004719
004720
004721
004722
004723
004724
004725
004726
004727
004728
004729
004730
004731
004732
004733
004734
004735
004736
004737
004738
004739
004740
004741
004742
004743
004744
004745
004746
004747
004748
004749
004750
004751
004752
004753
004754
004755
004756
004757
004758
004759
004760
004761
004762
004763
004764
004765
004766
004767
004768
004769
004770
004771
004772
004773
004774
004775
004776
004777
004778
004779
004780
004781
004782
004783
004784
004785
004786
004787
004788
004789
004790
004791
004792
004793
004794
004795
004796
004797
004798
004799
004800
004801
004802
004803
004804
004805
004806
004807
004808
004809
004810
004811
004812
004813
004814
004815
004816
004817
004818
004819
004820
004821
004822
004823
004824
004825
004826
004827
004828
004829
004830
004831
004832

...

018782
018783
018784
018785
018786
018787
018788
018789
018790
018791
018792
018793
018794
018795
018796
018797
018798
018799
018800
018801
018802
018803
018804
018805
018806
018807
018808
018809
018810
018811
018812
018813
018814
018815
018816
018817
018818
018819
018820
018821
018822
018823
018824
018825
018826
018827
018828
018829
018830
018831
018832
018833
018834
018835
018836
018837
018838
018839
018840
018841
018842
018843
018844
018845
018846
018847
018848
018849
018850
018851
018852
018853
018854
018855
018856
018857
018858
018859
018860
018861
018862
018863
018864
018865
018866
018867
018868
018869
018870
018871
018872
018873
018874
018875
018876
018877
018878
018879
018880
018881
018882
018883
018884
018885
018886
018887
018888
018889
018890
018891
018892
018893
018894
018895
018896
018897
018898
018899
018900
018901
018902
018903
018904
018905
018906
018907
018908
018909
018910
018911
018912
018913
018914
018915
018916
018917
018918
018919
018920
018921
018922
018923
018924

019956
019957
019958
019959
019960
019961
019962
019963
019964
019965
019966
019967
019968
019969
019970
019971
019972
019973
019974
019975
019976
019977
019978
019979
019980
019981
019982
019983
019984
019985
019986
019987
019988
019989
019990
019991
019992
019993
019994
019995
019996
019997
019998
019999
020000
020001
020002
020003
020004
020005
020006
020007
020008
020009
020010
020011
020012
020013
020014
020015
020016
020017
020018
020019
020020
020021
020022
020023
020024
020025
020026
020027
020028
020029
020030
020031
020032
020033
020034
020035
020036
020037
020038
020039
020040
020041
020042
020043
020044
020045
020046
020047
020048
020049
020050
020051
020052
020053
020054
020055
020056
020057
020058
020059
020060
020061
020062
020063
020064
020065
020066
020067
020068
020069
020070
020071
020072
020073
020074
020075
020076
020077
020078
020079
020080
020081
020082
020083
020084
020085
020086
020087
020088
020089
020090
020091
020092
020093
020094
020095
020096
020097
020098

021134
021135
021136
021137
021138
021139
021140
021141
021142
021143
021144
021145
021146
021147
021148
021149
021150
021151
021152
021153
021154
021155
021156
021157
021158
021159
021160
021161
021162
021163
021164
021165
021166
021167
021168
021169
021170
021171
021172
021173
021174
021175
021176
021177
021178
021179
021180
021181
021182
021183
021184
021185
021186
021187
021188
021189
021190
021191
021192
021193
021194
021195
021196
021197
021198
021199
021200
021201
021202
021203
021204
021205
021206
021207
021208
021209
021210
021211
021212
021213
021214
021215
021216
021217
021218
021219
021220
021221
021222
021223
021224
021225
021226
021227
021228
021229
021230
021231
021232
021233
021234
021235
021236
021237
021238
021239
021240
021241
021242
021243
021244
021245
021246
021247
021248
021249
021250
021251
021252
021253
021254
021255
021256
021257
021258
021259
021260
021261
021262
021263
021264
021265
021266
021267
021268
021269
021270
021271
021272
021273
021274
021275
021276

022309
022310
022311
022312
022313
022314
022315
022316
022317
022318
022319
022320
022321
022322
022323
022324
022325
022326
022327
022328
022329
022330
022331
022332
022333
022334
022335
022336
022337
022338
022339
022340
022341
022342
022343
022344
022345
022346
022347
022348
022349
022350
022351
022352
022353
022354
022355
022356
022357
022358
022359
022360
022361
022362
022363
022364
022365
022366
022367
022368
022369
022370
022371
022372
022373
022374
022375
022376
022377
022378
022379
022380
022381
022382
022383
022384
022385
022386
022387
022388
022389
022390
022391
022392
022393
022394
022395
022396
022397
022398
022399
022400
022401
022402
022403
022404
022405
022406
022407
022408
022409
022410
022411
022412
022413
022414
022415
022416
022417
022418
022419
022420
022421
022422
022423
022424
022425
022426
022427
022428
022429
022430
022431
022432
022433
022434
022435
022436
022437
022438
022439
022440
022441
022442
022443
022444
022445
022446
022447
022448
022449
022450
022451

023481
023482
023483
023484
023485
023486
023487
023488
023489
023490
023491
023492
023493
023494
023495
023496
023497
023498
023499
023500
023501
023502
023503
023504
023505
023506
023507
023508
023509
023510
023511
023512
023513
023514
023515
023516
023517
023518
023519
023520
023521
023522
023523
023524
023525
023526
023527
023528
023529
023530
023531
023532
023533
023534
023535
023536
023537
023538
023539
023540
023541
023542
023543
023544
023545
023546
023547
023548
023549
023550
023551
023552
023553
023554
023555
023556
023557
023558
023559
023560
023561
023562
023563
023564
023565
023566
023567
023568
023569
023570
023571
023572
023573
023574
023575
023576
023577
023578
023579
023580
023581
023582
023583
023584
023585
023586
023587
023588
023589
023590
023591
023592
023593
023594
023595
023596
023597
023598
023599
023600
023601
023602
023603
023604
023605
023606
023607
023608
023609
023610
023611
023612
023613
023614
023615
023616
023617
023618
023619
023620
023621
023622
023623

In [ ]:
row = detections_tracked_df.iloc[0]

In [ ]:
mask_count = int(row["n_count"]) * 2

In [ ]:
mask_count

In [ ]:
masks = row[9:mask_count+9].to_list()

In [ ]:
video_path

In [ ]:
import cv2
cap= cv2.VideoCapture(video_path.format(record_id, drive_id))
ret, frame = cap.read()
cv2.imwrite('test_prevention.jpg',frame)
cap.release()

In [ ]:
contours = []
for i in range(0,len(masks),2):
    contours.append([int(masks[i]),int(masks[i+1])])

In [ ]:
cntr = np.array(contours)

In [ ]:
colorr = tuple(np.random.randint(256, size=3))

In [ ]:
colr = (colorr[0], colorr[1], colorr[2])

In [ ]:
colr

In [ ]:
colr == colorr

In [ ]:
image = cv2.imread("test_prevention.jpg")
cv2.fillPoly(image, pts =[cntr], color=(int(colorr[0]), int(colorr[1]), int(colorr[2])))
"""
for pxl in contours:
    image[pxl[1],pxl[0]] = (255,255,0)
"""
cv2.imwrite("test_mask2.jpg", image)

- [ ] create a black image with the same size of frame
- [ ] put detected contour to the image with random color (remove selected color from color list)
- [ ] do this for each detected cars

In [ ]:
bbox = [row["xi"], row["yi"], row["xf"], row["yf"]]
bbox = bbox.astype(int)

label = row["veh_class"]
score = row["conf"]
mask_count = row["n_count"]

masks = row[9:,:mask_count]

In [ ]:
import cv2
def draw_mask(image, box, mask, label=None, color=None, binarize_threshold=0.5):
    """ Draws a mask in a given box.
    Args
        image              : Three dimensional image to draw on.
        box                : Vector of at least 4 values (x1, y1, x2, y2) representing a box in the image.
        mask               : A 2D float mask which will be reshaped to the size of the box, binarized and drawn over the image.
        color              : Color to draw the mask with. If the box has 5 values, the last value is assumed to be the label and used to construct a default color.
        binarize_threshold : Threshold used for binarizing the mask.
    Returns
        indices            : List of indices representing a mask.
    """
    if label is not None:
        color = label_color(label)
    if color is None:
        color = (0, 255, 0)

    # resize to fit the box
    mask = mask.astype(np.float32)
    mask = cv2.resize(mask, (box[2] - box[0], box[3] - box[1]))

    # binarize the mask
    mask = (mask > binarize_threshold).astype(np.uint8)

    # draw the mask in the image
    mask_image = np.zeros((image.shape[0], image.shape[1]), np.uint8)
    mask_image[box[1]:box[3], box[0]:box[2]] = mask
    mask = mask_image

    # compute a nice border around the mask
    border = mask - cv2.erode(mask, np.ones((5, 5), np.uint8), iterations=1)

    # apply color to the mask and border
    mask = (np.stack([mask] * 3, axis=2) * color).astype(ddnp.uint8)
    border = (np.stack([border] * 3, axis=2) * (255, 255, 255)).astype(np.uint8)

    # draw the mask
    indices = np.where(mask != [0, 0, 0])
    image[indices[0], indices[1], :] = 0.5 * image[indices[0], indices[1], :] + 0.5 * mask[indices[0], indices[1], :]

    # draw the border
    border_indices = np.where(border != [0, 0, 0])
    image[border_indices[0], border_indices[1], :] = 0.2 * image[border_indices[0], border_indices[1], :] + 0.8 * border[border_indices[0], border_indices[1], :]

    return indices

In [ ]:
def draw_masks(image, boxes, masks, labels=None, color=None, binarize_threshold=0.5):
    """ Draws a list of masks given a list of boxes.
    Args
        image              : Three dimensional image to draw on.
        boxes              : Matrix of shape (N, >=4) (at least 4 values: (x1, y1, x2, y2)) representing boxes in the image.
        masks              : Matrix of shape (N, H, W) of N masks of shape (H, W) which will be reshaped to the size of the corresponding box, binarized and drawn over the image.
        labels             : Optional list of labels, used to color the masks with. If provided, color is ignored.
        color              : Color or to draw the masks with.
        binarize_threshold : Threshold used for binarizing the masks.
    Returns
        indices            : List of lists of indices ; each list of indices represents a mask.
    """
    if labels is None:
        labels = [None for _ in range(boxes.shape[0])]

    indices = []
    for box, mask, label in zip(boxes, masks, labels):
        indices.append(draw_mask(image, box, mask, label=label, color=color, binarize_threshold=binarize_threshold))
    return indices

In [ ]:
def draw_box(image, box, color, thickness=2):
    """ Draws a box on an image with a given color.
    # Arguments
        image     : The image to draw on.
        box       : A list of 4 elements (x1, y1, x2, y2).
        color     : The color of the box.
        thickness : The thickness of the lines to draw a box with.
    """
    b = np.array(box).astype(int)
    cv2.rectangle(image, (b[0], b[1]), (b[2], b[3]), color, thickness, cv2.LINE_AA)

### To Do for each record and each drive
1. read lanechanges.txt 
2. if type is 0,1,3,4,5 add that to a new dataframe with record and drive name
3. crop video with using f0-60 and ff+60
4. save cropped video with name record_X_drivey_id__id_d_lc_type_blinker.mov

In [ ]:
lane_changes_df = pd.read_csv(data_info_path.format(record_id, drive_id, "lane_changes.txt"), sep=" ",
                              names=["id", "id_d", "lc_type", "f0", "fe", "ff", "blinker"])

In [ ]:
lane_changes_df.head(5)

In [ ]:
for idx, row in lane_changes_df.iterrows():
    from IPython.core.debugger import set_trace
    #set_trace()
    f0 = int(row['f0'])
    ff = int(row['ff'])
    t1 = (f0-10) / 10
    t2 = (ff+10) / 10
    _id = row["id"]
    id_d = row["id_d"]
    lc_type = row["lc_type"]
    blinker = row["blinker"]
    save_file =  "/media/yagiz/a4d0bbab-b26d-4047-901d-b736c2789e3f/prevention_dataset/record{}/drive{}/lane_changes/record{}_drive{}_{}_{}_{}_{}.mp4".format(record_id, drive_id, record_id, drive_id, _id, id_d, lc_type, blinker)
    print(save_file)
    ffmpeg_extract_subclip(video_path.format(record_id, drive_id), t1, t2, targetname=save_file)

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
t1 = (f0-10) / 10
t2 = (ff+10) / 10
#save_file =  "/media/yagiz/a4d0bbab-b26d-4047-901d-b736c2789e3f/prevention_dataset/record{}/drive{}/lane_changes/record{}_drive{}_{}_{}_{}_{}.mp4".format(record_id, drive_id, record_id, drive_id, _id, id_d, lc_type, blinker)
ffmpeg_extract_subclip(video_path.format(record_id, drive_id), t1, t2, targetname="test.mp4")